In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
fashion_mnist = keras.datasets.fashion_mnist.load_data()
(X_train, y_train), (X_test, y_test) = fashion_mnist

X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

X_train = X_train.reshape(-1,28,28,1)

4431872/4422102 [==============================] - 0s 0us/step


In [3]:
from functools import partial

In [6]:
DefaultConv2D = partial(keras.layers.Conv2D,strides=1,padding="SAME",kernel_size=3,use_bias=False)

In [7]:
# Creating Residual Block
class ResidualUnit(keras.layers.Layer):
    def __init__(self,filters,strides=1,activation='relu',**kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [DefaultConv2D(filters,strides=strides),keras.layers.BatchNormalization(),self.activation,
                            DefaultConv2D(filters),keras.layers.BatchNormalization()]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [DefaultConv2D(filters,kernel_size=1,strides=strides),keras.layers.BatchNormalization()]
        
    def call(self,inputs):
        Z = inputs
        for layers in self.main_layers:
            Z = layers(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
            
        return self.activation(Z+skip_Z)

In [8]:
model = keras.models.Sequential()
#Conv layer
model.add(DefaultConv2D(64,kernel_size=7,strides=2,input_shape=[28,28,1]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPooling2D(pool_size=3,strides=2,padding="SAME"))
#Deep Residual Network
prev_filters = 64
for filters in [64]*2 + [128]*2 + [256]*2 + [512]*2 :
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters,strides=strides))
    prev_filters = filters
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10,activation="softmax"))

In [9]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

In [10]:
model.fit(X_train,y_train,epochs=15)

Epoch 1/15
1875/1875 [==============================] - 45s 17ms/step - loss: 0.4789 - accuracy: 0.8297
Epoch 2/15
1875/1875 [==============================] - 32s 17ms/step - loss: 0.3041 - accuracy: 0.8867
Epoch 3/15
1875/1875 [==============================] - 32s 17ms/step - loss: 0.2556 - accuracy: 0.9052
Epoch 4/15
1875/1875 [==============================] - 33s 17ms/step - loss: 0.2261 - accuracy: 0.9155
Epoch 5/15
1875/1875 [==============================] - 33s 17ms/step - loss: 0.2013 - accuracy: 0.9238
Epoch 6/15
1875/1875 [==============================] - 32s 17ms/step - loss: 0.1815 - accuracy: 0.9306
Epoch 7/15
1875/1875 [==============================] - 32s 17ms/step - loss: 0.1621 - accuracy: 0.9389
Epoch 8/15
1875/1875 [==============================] - 33s 17ms/step - loss: 0.1458 - accuracy: 0.9440
Epoch 9/15
1875/1875 [==============================] - 33s 18ms/step - loss: 0.1314 - accuracy: 0.9500
Epoch 10/15
1875/1875 [==============================] - 33s 17m

In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 64)        3136      
                                                                 
 batch_normalization (BatchN  (None, 14, 14, 64)       256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 14, 14, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 7, 7, 64)         0         
 )                                                               
                                                                 
 residual_unit (ResidualUnit  (None, 7, 7, 64)         74240     
 )                                                               
                                                      